In [4]:
import pandas as pd
import duckdb 
import sqlite3
from datetime import datetime
import numpy as np


In [2]:
database_name ="../test_analytics.db"


In [ ]:
def returning_df_from_all_tables(query):
	try:

		conn = sqlite3.connect(database_name)
		cursor = conn.cursor()

		cursor.execute (query)
		tables = [table[0] for table in cursor.fetchall()]

		if not tables:
			print ("Not is possible")

		union_all_query = " UNION ALL ".join(f"SELECT * FROM \"{table}\"" for table in tables)
		#cursor.execute(union_all_query)

		# rows = cursor.fetchall()
		df = pd.read_sql_query(union_all_query, conn)

		return df

	except sqlite3.Error as e:
		print(f"SQLITE error: {e}")

	finally:
		if conn:
			conn.close()


query =   """
  SELECT name FROM sqlite_master
  WHERE type = 'table' AND  (name LIKE 'escolas%' OR name LIKE '%escolas%');
  """

query2 = """
  SELECT name FROM sqlite_master
  WHERE type = 'table' AND  (name LIKE 'idadeserie%' OR name LIKE '%idadeserie%');
  """

df_escolas = returning_df_from_all_tables(query)

df_estudantes = returning_df_from_all_tables(query2)

In [5]:
duckdb_database = "../my_database.duckdb"

In [ ]:
conn = duckdb.connect(duckdb_database)

In [ ]:
conn.execute("CREATE TABLE IF NOT EXISTS raw_escolas AS SELECT * FROM  df_escolas")

In [ ]:
conn.execute("CREATE TABLE IF NOT EXISTS raw_estudantes AS SELECT * FROM df_estudantes")

In [9]:
escola = conn.sql(
  """
    SELECT DISTINCT
      re.dre,
      re.codesc,
      re.tipoesc,
      re.nomesc,
      re.diretoria,
      re.fx_etaria,
      re.situacao,
      re.database,
      re.dt_criacao ,
      re.dt_ini_conv ,
      re.cep
    FROM main.raw_escolas re
    WHERE re.fx_etaria IS NOT NULL
    ORDER BY re.codesc;
  """
  )

In [10]:
endereco = conn.sql(
  """
    SELECT DISTINCT
      re.endereco ,
      re.bairro ,
      re.distrito,
      re.coddist,
      re.cep,
      re.latitude ,
      re.longitude ,
      
    FROM main.raw_escolas re
    WHERE re.fx_etaria IS NOT NULL
    ORDER BY re.codesc;
  """
  )

In [11]:
estudantes = conn.sql(

  """
    SELECT * FROM main.raw_estudantes
  
  """

)

In [12]:
df = escola.to_df()

In [13]:
df_endereco = endereco.to_df()

In [14]:
df_estudantes = estudantes.to_df()

In [15]:
print(df_estudantes.shape)

(2982929, 19)


# Processing and EDA df escolas

In [16]:
categorical_columns = df.select_dtypes(include=['object', 'category']).columns
date_columns = ["dt_criacao","dt_ini_func", "dt_ini_conv","dt_autoriza","database"]

In [17]:
categorical_columns = categorical_columns.difference(date_columns)

In [18]:
# Upper All columns
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.str.upper())


In [19]:
# Change from ',' to '.' and strip for nomeesc
df["nomesc"] = df["nomesc"].str.replace(",", "")
df["nomesc"] = df["nomesc"].str.strip()

In [20]:
df.head()

,dre,codesc,tipoesc,nomesc,diretoria,fx_etaria,situacao,database,dt_criacao,dt_ini_conv,cep
0,PJ,30993,CR.P.CONV,VILA SULINA,PIRITUBA,00 A 03,ATIVA,31/12/2020 00:00,07/03/2020 00:00,None,5264050
1,PJ,200204,CECI,JARAGUA,PIRITUBA,ZERO A 6A E 11M,ATIVA,27/12/2012,19/02/2004,23/12/2004,5177100
2,PJ,200204,CECI,JARAGUA,PIRITUBA,ZERO A 5 A,ATIVA,30/11/2013,19/02/2004,23/12/2004,5177100
3,PJ,200204,CECI,JARAGUA,PIRITUBA,ZERO A 6A E 11M,ATIVA,31/12/2010,19/02/2004,23/12/2004,5177100
4,PJ,200204,CECI,JARAGUA,PIRITUBA,ZERO A 6A E 11M,ATIVA,31/12/2011,19/02/2004,23/12/2004,5177100


In [21]:
month_map = {
    "jan": "01", "fev": "02", "mar": "03", "abr": "04",
    "mai": "05", "jun": "06", "jul": "07", "ago": "08",
    "set": "09", "out": "10", "nov": "11", "dez": "12"
}

In [22]:
def translate_brazilian_month(date_str):
    """Replace Brazilian month abbreviations with numeric equivalents."""
    if not isinstance(date_str, str):  # Check if the value is a string
        return date_str  # Return the original value (e.g., None)
    
    for pt_month, num_month in month_map.items():
        if pt_month in date_str:
            return date_str.replace(pt_month, num_month)
    return date_str


In [23]:
def parse_date(date_str):
      """Try to parse the date string into a standardized format."""
      date_str = date_str.strip().lower()
      formats = [
          "%d/%m/%Y",    # DD/MM/YYYY
          "%d/%m/%y",    # DD/MM/YY
          "%d/%b/%y",    # DD/Mon/YY (abbreviated month)
          "%d/%m/%Y %H:%M"  # DD/MM/YYYY HH:MM
      ]
      for fmt in formats:
          try:
              return datetime.strptime(date_str, fmt).date()
          except ValueError:
                continue
      return None  # Return None if parsing fails


In [24]:
df = df.dropna(subset=['database', "dt_criacao"])
df["dt_ini_conv"] = df["dt_ini_conv"].fillna("01/01/2099")

In [25]:
data_cleaned = df.copy()

In [26]:
data_cleaned['dt_criacao'] = (
    data_cleaned['dt_criacao']
    .apply(translate_brazilian_month)  # Replace Brazilian months
    .apply(parse_date)  # Parse the dates
)

data_cleaned['database'] = (
    data_cleaned['database']
    .apply(translate_brazilian_month)  # Replace Brazilian months
    .apply(parse_date)  # Parse the dates
)

data_cleaned['dt_ini_conv'] = (
    data_cleaned['dt_ini_conv']
    .apply(translate_brazilian_month)  # Replace Brazilian months
    .apply(parse_date)  # Parse the dates
)

In [27]:
#data_cleaned["dt_criacao"] = pd.to_datetime(data_cleaned["dt_criacao"], format="%Y-%m")
data_cleaned['database'] = pd.to_datetime(data_cleaned['database']).dt.to_period('M').astype(str)
data_cleaned['dt_criacao'] = pd.to_datetime(data_cleaned['dt_criacao']).dt.to_period('M').astype(str)
data_cleaned['dt_ini_conv'] = pd.to_datetime(data_cleaned['dt_ini_conv']).dt.to_period('M').astype(str)


In [28]:
#df = df.drop_duplicates(subset=['codesc','nomesc','dt_criacao'])
data_cleaned = data_cleaned.drop_duplicates(subset=['codesc','nomesc','dt_criacao'])

# Processing and EDA endereco

In [33]:
aux = df_endereco.drop_duplicates(subset=["cep","latitude","longitude"])

In [34]:
mask = aux["cep"] == 2860001

In [35]:
aux[mask]

,endereco,bairro,distrito,coddist,cep,latitude,longitude
964,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-46688471.0,-23459785.0
965,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-23459785,-46688471
966,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-23459785.0,-46688471.0
967,Avenida DEPUTADO CANTÃDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-23.459.785,-46.688.471
968,DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-23451864,-46692602
969,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,BRASILANDIA,11.0,2860001,-23458827,-46686326
1396,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,JARAGUA,42.0,2860001,-23435657,-46716572
1397,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,JARAGUA,42.0,2860001,-23435682,-46715909
1398,Avenida DEPUTADO CANTÃDIO SAMPAIO,VILA SOUZA,JARAGUA,42.0,2860001,-23.435.682,-46.715.909
1399,AVENIDA DEPUTADO CANTÍDIO SAMPAIO,VILA SOUZA,JARAGUA,42.0,2860001,-23435682.0,-46715909.0


In [36]:
def corrigir_coordenadas(df):
    for index, row in df.iterrows():
        try:
            lat = float(str(row['latitude']).replace('.', '').replace(',', '.'))
            lon = float(str(row['longitude']).replace('.', '').replace(',', '.'))

            if abs(lat) > 90 or abs(lon) > 180:
                lat, lon = lon, lat

            df.loc[index, 'latitude'] = round(lat, 6)
            df.loc[index, 'longitude'] = round(lon, 6)

        except (ValueError, TypeError):
            print(f"Erro na linha {index}: Latitude: {row['latitude']}, Longitude: {row['longitude']}")
            df.loc[index, 'latitude'] = np.nan
            df.loc[index, 'longitude'] = np.nan

    return df

# df = corrigir_coordenadas(df)
# print(df[['latitude', 'longitude']].drop_duplicates())

In [37]:
aux = aux.dropna(subset=['latitude','longitude'])
print(aux.shape)

(11864, 7)


In [38]:
aux.sort_values('latitude').head(10)

,endereco,bairro,distrito,coddist,cep,latitude,longitude
991,Avenida PERI RONCHETTI,JARDIM PERI,CACHOEIRINHA,13.0,2633000,#CAMPO!,-46.648.435
2004,PEDRO VOSS,VILA CARRÃO,CARRAO,20.0,3437000,-140000000,-46531496
5564,Rua PRESIDENTE VARGAS,VILA CAIÃÅ¡BA,PERUS,61.0,5207000,-2.340.161,-46.748.203
8794,Rua TABULEIROS DE VALENCA,VILA INÃCIO,PERUS,61.0,5206170,-2.340.424,-46.765.869
8903,Rua MOGEIRO,VILA PERUS,PERUS,61.0,5208230,-2.340.555,-46.759.129
11105,Rua TANAGRA VIOLÃÂCEA,JARDIM RODRIGO,JARAGUA,42.0,2990350,-2.342.889,-4.672.868
8656,Rua LÃÂBERO TEIXEIRA BRAGA,VILA AURORA,JARAGUA,42.0,5186110,-2.343.609,-46.750.823
1390,Rua MONTE AZUL PAULISTA,VILA NOVA PARADA,JARAGUA,42.0,2883050,-2.343.984,-46.711.776
1395,Rua MONTE AZUL PAULISTA,VILA NOVA PARADA,JARAGUA,42.0,2883050,-2.344.018,-46.712.188
14690,Rua ILHA DA JUVENTUDE,VILA BRASILÃâNDIA,JARAGUA,42.0,2820000,-2.344.522,-46.708.955


In [39]:
aux['endereco'] = aux['endereco'].str.replace('Ã', 'Í').str.replace('ÃƒÂ', 'Í').str.replace('Ã', 'A').str.strip()
aux['endereco'] = aux['endereco'].str.replace('Avenida', 'AVENIDA', case=False)
aux['endereco'] = aux['endereco'].str.replace(r'\s+', ' ', regex=True).str.strip()
aux = corrigir_coordenadas(aux)
df_limpo = aux.drop_duplicates(subset=['endereco', 'cep', 'latitude', 'longitude'], keep='first')

print(df_limpo)

Erro na linha 991: Latitude: #CAMPO!, Longitude: -46.648.435
Erro na linha 9868: Latitude: -23.562.178, Longitude: #CAMPO!
                                endereco                               bairro  \
0                   RUA LUÍSA MARIA ROSA                          VILA SULINA   
1       AVENIDA COMENDADOR JOSE DE MATOS                         VILA CLARICE   
2       AVENIDA COMENDADOR JOSE DE MATOS                         VILA CLARICE   
4       AVENIDA COMENDADOR JOSE DE MATOS                         VILA CLARICE   
5       AVENIDA COMENDADOR JOSE DE MATOS                         VILA CLARICE   
...                                  ...                                  ...   
14825                    RUA MERIDIONAIS  JARDIM SANTA TEREZINHA (ZONA LESTE)   
14826           RUA LIRA DOS VERDES ANOS                   JARDIM SÃO SAVÉRIO   
14827             RUA DOUTOR MILTON PENA               PARQUE CRUZEIRO DO SUL   
14828    RUA MAJOR JOSÉ MARIOTO FERREIRA                         PA

In [47]:
df_limpo = df_limpo.drop_duplicates(subset='endereco')

In [ ]:
df_limpo.drop_duplicates(subset='cep')

# EDA and Processing estudantes

In [40]:
df_estudantes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2982929 entries, 0 to 2982928
Data columns (total 19 columns):
 #   Column     Dtype  
---  ------     -----  
 0   dre        object 
 1   codesc     int64  
 2   tipoesc    object 
 3   nomesc     object 
 4   distrito   object 
 5   setor      int64  
 6   ano        int64  
 7   rede       object 
 8   modal      object 
 9   descserie  object 
 10  periodo    object 
 11  turno      int64  
 12  descturno  object 
 13  sexo       object 
 14  idade      float64
 15  nee        object 
 16  raca       object 
 17  qtde       int64  
 18  database   object 
dtypes: float64(1), int64(5), object(13)
memory usage: 432.4+ MB


In [41]:
df_estudantes = df_estudantes.dropna(axis=0)

In [42]:
df_estudantes["tipoesc"].unique()

array(['EMEF', 'EMEI', 'CEU EMEF', 'CEU EMEI', 'CEMEI', 'CIEJA',
       'CR.P.CONV', 'CEI INDIR', 'CEI DIRET', 'CEU CEI', 'MOVA',
       'ESP CONV', 'CEU AT COM', 'CECI', 'EMEBS', 'CCI/CIPS', 'EMEFM',
       'E TECNICA', 'CMCT', 'EMEF        ', 'EMEI        ',
       'CEU EMEF    ', 'CEU EMEI    ', 'CEMEI       ', 'CIEJA       ',
       'CR.P.CONV   ', 'CEI INDIR   ', 'CEI DIRET   ', 'CEU CEI     ',
       'ESP CONV    ', 'CEU AT COMPL', 'MOVA        ', 'CECI        ',
       'EMEBS       ', 'CCI/CIPS    ', 'EMEFM       ', 'CCA', 'CEU CEMEI',
       'E T', 'E TEC'], dtype=object)

In [43]:
df_estudantes["tipoesc"] = df_estudantes["tipoesc"].str.strip()

C:\Users\davi_\AppData\Local\Temp\ipykernel_30852\3200536890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_estudantes["tipoesc"] = df_estudantes["tipoesc"].str.strip()


In [44]:
df_estudantes["tipoesc"].unique()

array(['EMEF', 'EMEI', 'CEU EMEF', 'CEU EMEI', 'CEMEI', 'CIEJA',
       'CR.P.CONV', 'CEI INDIR', 'CEI DIRET', 'CEU CEI', 'MOVA',
       'ESP CONV', 'CEU AT COM', 'CECI', 'EMEBS', 'CCI/CIPS', 'EMEFM',
       'E TECNICA', 'CMCT', 'CEU AT COMPL', 'CCA', 'CEU CEMEI', 'E T',
       'E TEC'], dtype=object)

In [45]:
df_estudantes = df_estudantes.drop_duplicates()
print(df_estudantes.shape)

(2586757, 19)


In [46]:
np.sort(df_estudantes["modal"].unique())

array(['ATCOMP', 'BERCARIO', 'CELP', 'CONVEE', 'CRECHE', 'EDPROF',
       'EDUCACAO INFANTIL', 'EJA', 'EJA CIEJA',
       'EJA ESCOLAS EDUCACAO ESPECIAL', 'EJA ESCOLAS ENSINO FUNDAMENTAL',
       'ENSINO FUNDAMENTAL 9 ANOS', 'ENSINO FUNDAMENTAL DE 9 ANOS',
       'ENSINO MEDIO', 'ENSINO MEDIO NORMAL/MAGISTERIO', 'ESPEC',
       'ESPEC ENS MEDIO', 'FUND', 'MEDIO', 'MOVA', 'OIE', 'OSL', 'PRE',
       'PROJ', 'REC', 'SAAI', 'TECNICO MEDIO'], dtype=object)

In [47]:
mask = df_estudantes["modal"] == "CELP"
df_estudantes[mask].head()

,dre,codesc,tipoesc,nomesc,distrito,setor,ano,rede,modal,descserie,periodo,turno,descturno,sexo,idade,nee,raca,qtde,database
2772429,CL,700002,CEU AT COM,"CASA BLANCA - SOLON BORGES DOS REIS, PROF.",JARDIM SAO LUIS,4601,2022,DIR,CELP,CELP ESPANHOL,DIURNO,1,Manhã,F,11.0,NAO POSSUI,PARDA,1,30/12/2022 00:00
2772430,CL,700002,CEU AT COM,"CASA BLANCA - SOLON BORGES DOS REIS, PROF.",JARDIM SAO LUIS,4601,2022,DIR,CELP,CELP ESPANHOL,DIURNO,1,Manhã,M,10.0,NAO POSSUI,BRANCA,1,30/12/2022 00:00
2772431,CL,700002,CEU AT COM,"CASA BLANCA - SOLON BORGES DOS REIS, PROF.",JARDIM SAO LUIS,4601,2022,DIR,CELP,CELP ESPANHOL,DIURNO,1,Manhã,M,12.0,NAO POSSUI,PARDA,1,30/12/2022 00:00
2772432,CL,700002,CEU AT COM,"CASA BLANCA - SOLON BORGES DOS REIS, PROF.",JARDIM SAO LUIS,4601,2022,DIR,CELP,CELP ESPANHOL,DIURNO,1,Manhã,M,13.0,NAO POSSUI,BRANCA,1,30/12/2022 00:00
2772433,CL,700002,CEU AT COM,"CASA BLANCA - SOLON BORGES DOS REIS, PROF.",JARDIM SAO LUIS,4601,2022,DIR,CELP,CELP ESPANHOL,DIURNO,1,Manhã,M,13.0,NAO POSSUI,PARDA,1,30/12/2022 00:00


In [48]:
df_estudantes.value_counts('descturno')

descturno
Tarde                      544301
Manhã                      514659
Integral                   505859
Tarde                      247711
Integral                   204153
Noite                      178756
ManhÃ£                     159481
Noite                      100132
ManhÃÂ£                    79961
Intermediário               20248
Vespertino                  15120
IntermediÃ¡rio               8415
Vespertino                   6515
IntermediÃÂ¡rio             1446
Name: count, dtype: int64

In [49]:
novos_modais = {
  "BERCARIO": "CRECHE",
  "CELP": "ATCOMP",
  "EDUCACAO INFANTIL": "PRE",
  "EJA CIEJ": "EJA",
  "EJA CIEJA": "EJA",
  "EJA ESCOLAS EDUCACAO ESPECIAL":"EJA",
  "EJA ESCOLAS ENSINO FUNDAMENTAL":"EJA",
  "ENSINO FUNDAMENTAL 9 ANOS": "FUND",
  "ENSINO FUNDAMENTAL DE 9 ANOS": "FUND",
  "ENSINO MEDIO": "MEDIO",
  "ENSINO MEDIO NORMAL/MAGISTERIO": "MEDIO",
  "ESPEC ENS MEDIO": "MEDIO",
  "TECNICO MEDIO": "MEDIO"
}

In [50]:
df_estudantes["modal"] = df_estudantes["modal"].replace(novos_modais)

In [51]:
np.sort(df_estudantes["modal"].unique())

array(['ATCOMP', 'CONVEE', 'CRECHE', 'EDPROF', 'EJA', 'ESPEC', 'FUND',
       'MEDIO', 'MOVA', 'OIE', 'OSL', 'PRE', 'PROJ', 'REC', 'SAAI'],
      dtype=object)

In [52]:
df_estudantes["descturno"].unique()

array(['Tarde', 'Manhã', 'Intermediário', 'Noite', 'Integral',
       'Vespertino', 'Tarde               ', 'ManhÃ\x83Â£               ',
       'Noite               ', 'Integral            ',
       'IntermediÃ\x83Â¡rio       ', 'Vespertino          ',
       'ManhÃ£               ', 'IntermediÃ¡rio       '], dtype=object)

In [ ]:
try:
    unique_series = np.sort(df_estudantes['descserie'].str.encode('latin1').str.decode('utf-8').unique())
except UnicodeDecodeError:
    try:
        unique_series = np.sort(df_estudantes['descserie'].str.encode('cp1252').str.decode('utf-8').unique())
    except UnicodeDecodeError:
        # Try other encodings here (e.g., iso-8859-1)
        unique_series = np.sort(df_estudantes['descserie'].str.encode('iso-8859-1').str.decode('utf-8').unique())

In [714]:
conn.close()